### Многопоточность. Основы Lock free.

Документация:
* https://en.cppreference.com/w/cpp/atomic
* https://en.cppreference.com/w/cpp/atomic/memory_order

Серия статей от Jeff Pershing на понимание atomics, memory model && lock free:
* https://preshing.com/20120515/memory-reordering-caught-in-the-act/
* https://preshing.com/20120612/an-introduction-to-lock-free-programming
* https://preshing.com/20120625/memory-ordering-at-compile-time/
* https://preshing.com/20120710/memory-barriers-are-like-source-control-operations/
* https://preshing.com/20120913/acquire-and-release-semantics/
* https://preshing.com/20120930/weak-vs-strong-memory-models/
* https://preshing.com/20121019/this-is-why-they-call-it-a-weakly-ordered-cpu/
* https://preshing.com/20130618/atomic-vs-non-atomic-operations/
* https://preshing.com/20130702/the-happens-before-relation/
* https://preshing.com/20130922/acquire-and-release-fences/
* https://preshing.com/20130823/the-synchronizes-with-relation/
* https://preshing.com/20130930/double-checked-locking-is-fixed-in-cpp11/
* https://preshing.com/20131125/acquire-and-release-fences-dont-work-the-way-youd-expect/
* https://preshing.com/20140709/the-purpose-of-memory_order_consume-in-cpp11/
* В конце своих статей Джефф даёт ссылки на полезные материалы, их тоже рекомендуется почитать. Не стоит рассчитывать, что управитесь со всем багажом знаний за полчасика.
* И в комментарии к статьям приходят специалисты (Herb Sutter, Tarvis Downs), и объясняют, в чём Джефф был не прав, поэтому комментарии желательно тоже смотреть.

Другие статьи:
* [Memory Barriers: a Hardware View for Software Hackers](http://www.rdrop.com/users/paulmck/scalability/paper/whymb.2010.07.23a.pdf)
* [LINUX KERNEL MEMORY BARRIERS](https://www.kernel.org/doc/Documentation/memory-barriers.txt)
* [Максим Хижинский. Lock-free Data Structures. Basics: Atomicity and Atomic Primitives](https://kukuruku.co/post/lock-free-data-structures-basics-atomicity-and-atomic-primitives/)
* [Bjarne Stroustrup про ABA-проблему и как её решать](http://www.stroustrup.com/isorc2010.pdf)

Доклады про многопоточность:
* [CppCon 2017: Fedor Pikus “C++ atomics, from basic to advanced. What do they really do?”](https://www.youtube.com/watch?v=ZQFzMfHIxng)
* [CppCon 2018: Bryce Adelstein Lelbach “The C++ Execution Model”](https://www.youtube.com/watch?v=FJIn1YhPJJc)
* [CppCon 2019: Bryce Adelstein Lelbach “The C++20 Synchronization Library”](https://www.youtube.com/watch?v=Zcqwb3CWqs4)
* [C++ Siberia 2019: Максим Хижинский, Hazard Pointer внутри и снаружи](https://www.youtube.com/watch?v=aczfcRhXrzI)

<br />

##### intro to lock free programming

https://preshing.com/20120612/an-introduction-to-lock-free-programming
    
Jeff Pershing даёт следующее определение lock-free:

> lock free programming = multithreaded app + shared memory + threads do not lock each other

Под "threads do not lock each other" понимается, что при любом причудливом планировании распределения фреймов cpu по потокам работа будет продвигаться. Если даже ОС полностью усыпила какой-то один поток, остальные продолжат выполнять работу по алгоритму.

**Вопрос**: если принять такое определение, почему любой алгоритм с использованием mutex не является lock free?

<br />

**Пример:** пример без мьютексов с атомарным `х` равным 0 на входе, который НЕ lock free:
    
```c++
while (X == 0)
    X = 1 - X;
```

**Вопрос:** каким образом два апотока могут здесь друг друга залочить

**Следствие:** отсутствие `mutex`-ов и `spin_lock`-ов ещё не делает алгоритм lock free

<br />

Другое определение lock free:
    
> as long as the program is able to keep calling those lock-free operations, the number of completed calls keeps increasing, no matter what

<br />

Если по какой-то причине после сегодняшней лекции (её примеров и упрощений) вам покажется, что lock-free - простая тема, [здесь](https://preshing.com/20120913/acquire-and-release-semantics/#IDComment721195803) пример как Jeff Pershing находит ошибку в презентации про атомики и барьеры от Герба Саттера (генсека <s>КПСС</s> КСС++, второго человека в мире С++).

<br />

##### hardware memory model (simplified)

https://preshing.com/20120930/weak-vs-strong-memory-models/

memory ordering - правила упорядочивания операций с памятью можно разделить на:

* правила языка для абстрактной машины
    * и вы программу пишите с ними и только под них
 
* hardware-специфичные правила
    * применяются в зависимости от железа, на котором выполняется программа - под них программу писать не надо, но желательно понимать, чтобы понимать, каких ошибок можно ожидать от железа, когда в программе наошибались в правилах языка

Поговорим про hardware memory ordering

Варианты:
* really weak
* weak with data dependency
* usually strong
* sequentially consistent

![](weak-strong-table-4.png)

**really weak**

(Без специальных барьеров) нет никаких гарантий:

Вспомним пример:

```c++
int x = 0, y = 0;
int a = 0, b = 0;

void thread_1_worker_on_cpu_1() {
    x = 1; y = 2;

    if (b == 5)
        assert(a == 4);  // fail
}

void thread_2_worker_on_cpu_2() {
    a = 4; b = 5;

    if (y == 2)
        assert(x == 1);  // fail
}
```

Нет никаких гарантий на порядок/время записи данных в "общую память (L2 + RAM)", ровно как и на порядок/время их просачивания до "частной памяти ЦПУ (L1)".


**weak with data dependency**

Появляется гарантия (из статьи Джеффа):

> It means that if you write A->B in C/C++, you are always guaranteed to load a value of B which is at least as new as the value of A

на примере:

```c++
int x = 0, y = 0;
int a = 0, b = 0;

void thread_1_worker_on_cpu_1() {
    x = 1;
    y = x;  // <---- в одну ячейку памяти пишем содержимое другой
    
    a = 1;
    b = 1;  // больше x, y, a, b никто не трогает
}

void thread_2_worker_on_cpu_2() {
    if (y == 1)
        assert(x == 1);  // ok

    if (b == 1)
        assert(a == 1);  // fail
}
```


**strong model**

> A strong hardware memory model is one in which every machine instruction comes implicitly with acquire and release semantics. As a result, when one CPU core performs a sequence of writes, every other CPU core sees those values change in the same order that they were written.

**Замечание**: Несмотря на то, что к strong model бодро отнесён x86/64, не все операции в x86/64 поддерживают strong memory model, читайте исходную статью и комментарии.

**Замечание**: ниже написан код плюсовый, но сделаем вид, будто компилятор ничего переупорядочивать не будет, если будет - всё неверно. Мы как будто бы пишем на ассемблере в синтаксисе плюсов.

```c++
int x = 0, y = 0, z = 0;

void thread_1_worker_on_cpu_1() {
    x = 1;
    y = 2;
    z = 3;
}

void thread_2_worker_on_cpu_2() {
    if (z == 3) {
        assert(y == 2);  // ok
        assert(x == 1);  // ok
    }
}
```

В strong модели разрешено переупорядочивание операций store-load (запись-чтение) (но запрещены load-store, store-store && load-load)

```c++
int x = 0, y = 0;
int a = 0, b = 0;

void thread_1_worker_on_cpu_1() {
    x = 1;  // store x
    a = y;  // load y
            // store a
}

void thread_2_worker_on_cpu_2() {
    y = 1;  // store y
    b = x;  // load x
            // store b
}
```

В результате такого "ассемблера" может получиться `a = 0 && b = 0`.

**Вопрос**: как?

<details>
<summary>ответ</summary>

* cpu1: register1 = read y ( == 0)
* cpu2: register2 = read x ( == 0)
* cpu1: x = 1
* cpu2: y = 1
* cpu1: a = register1 ( == 0)
* cpu2: b = register2 ( == 0)
    
    
</details>


**sequential consistent**

Любое переупорядочивание запрещено, из примера выше `a = 0 && b = 0` не получить.

<br />

##### C++ memory model (simplified, since C++11)

https://en.cppreference.com/w/cpp/atomic/memory_order

https://preshing.com/20120913/acquire-and-release-semantics

Теперь поговорим про memory ordering на уровне языка С++ - те, с которыми вам необходимо работать при написании программ.

В стандартной библиотеке С++ определён enum для различного рода ограничений на переупорядочивание операций с памятью.

```c++
typedef enum memory_order {
    memory_order_relaxed,
    memory_order_consume,
    memory_order_acquire,
    memory_order_release,
    memory_order_acq_rel,
    memory_order_seq_cst
} memory_order;
```

Из них рассмотрим:

```c++
memory_order_relaxed
memory_order_acquire
memory_order_release
memory_order_seq_cst
```

<br />

`memory_order_release`:

> A store operation with this memory order performs the release operation: no reads or writes in the current thread can be reordered after this store. 

```c++
a = 5;  // store a
x = 1;  // store x
y = a;  // load a  <== 5
        // store y

atomic_var.store(42, std::memory_order_release);
/*store op release barrier*/

... // операции, написанные выше с x, y и a
    // гарантировано будут выполнены до записи в atomic_var
```

`memory_order_acquire`:

> A load operation with this memory order performs the acquire operation on the affected memory location: no reads or writes in the current thread can be reordered before this load.

```c++
...  // операции, написанные ниже с x, y и a
     // не могут по времени произойти раньше чтения atomic_var
    
int z = atomic_var.load(std::memory_order_acquire);
/*load op acquire barrier*/

if (z == 42)
{
    x;  // load x  <== 1
    y;  // load y  <== 5
}
```

**Пример:** разобрать очень подробно про гарантии порядка вычислений. Вопрос: почему assert корректен?

```c++
std::atomic<bool> is_published{false};
int data = 0;

void thread_1_worker()
{
    data = 1;
    is_published.store(true, std::memory_order_release);
}

void thread_2_worker()
{
    if (is_published.load(std::memory_order_acquire))
        assert(data == 1);  // ok
}
```

Имена `release` и `acquire` подобраны по смыслу операции:
* `release` - опубликовать данные
* `acquire` - принять опубликованные данные

<br />

`memory_order_seq_cst` - операции не могут перескочить через барьер ни в какую сторону (аналог sequential consistent hardware memory model) - самый строгий и самый медленный вариант

`memory_order_seq_cst`:
* дефолтный способ упорядочивания в `atomic`-операциях (чтобы программисты меньше ошибались)
* аналог `volatile` в Java 5+



```c++
x = 1;  // store x
y = a;  // load a
        // store y

/*seq_cst barrier*/  // никакая из операций не может перескочить через барьер

z = 2;  // store x
b = c;  // load c
        // store b
```

<br />

`memory_order_relaxed`:
* возможны любые переупорядочивания операций с памятью (нет барьеров)
* гарантируется только атомарная модификация защищённого объекта

**Пример:**

```c++
std::atomic<int> atomic_var{0};

// нет гарантий на порядок операций с памятью,
// только атомарность работы с atomic_var
void thread_worker() {
    x = 1;
    atomic_var.store(2, std::memory_order_relaxed); 
    y = a;
}
```

<br />

Разница в гарантиях порядка необходима из-за различной стоимости барьеров в CPU: чем строже гарантии, тем медленнее могут выполняться команды.

* `relaxed` - самый быстрый (нет барьеров, только атомарность операции)
* `acquire/release` - медленнее или аналогично `relaxed` (барьеры в одну сторону + атомарность операции)
* `seq_cst` - медленнее или аналогично `acquire/release` (барьеры в обе стороны + атомарность операции)

<br />

##### оптимизированный DCLP

Напомню DCLP, на котором мы закончили его рассмотрение:

```c++
class Singleton
{
    static std::atomic<Singleton*> object;
    static std::mutex mtx;

public:
    static Singleton* instance()
    {
        auto* tmp = object.load();

        if (!tmp)  // check 1
        {
            std::lock_guard<std::mutex> lock(mtx);

            tmp = object.load();
            if (!tmp)  // check 2
            {
                tmp = new Singleton;
                object.store(tmp);
            }
        }
        return tmp;
    }
};
```

**Вопрос:** как, зная про memory ordering, сделать более быструю реализацию?

**Ответ:**

```c++
class Singleton
{
    static std::atomic<Singleton*> object;
    static std::mutex mtx;

public:
    static Singleton* instance()
    {
        auto* tmp = object.load(std::memory_order_acquire);  // <--- mem order on read data

        if (!tmp)  // check 1
        {
            std::lock_guard<std::mutex> lock(mtx);

            tmp = object.load(std::memory_order_acquire);  // <--- mem order on read data
            if (!tmp)  // check 2
            {
                tmp = new Singleton;
                object.store(tmp, std::memory_order_release);  // <--- mem order on publish data
            }
        }
        return tmp;
    }
};
```

**Замечание**: если верить [статье Jeff Pershing про `memory_order_consume`](https://preshing.com/20140709/the-purpose-of-memory_order_consume-in-cpp11/), в коде выше можно заменить `memory_order_acquire` на `memory_order_consume`, т.к. между load-операцией и дальнейшим кодом есть зависимость по данным, и всё станет ещё быстрее. Подробнее - читайте по ссылке.

<br />

##### оптимизированная сумма элементов в массиве

Напомню как мы считали сумму элементов в массиве:
    
```c++
int parallel_sum(const std::vector<int>& v, int threads_count)
{
    const int len = v.size() / threads_count;
    assert(len * threads_count == v.size());

    std::atomic<int> rv{0};  // <-- result

    std::vector<std::thread> threads;
    for (int i = 0; i < threads_count; ++i)
        threads.emplace_back([&, i, len](){
            for (int ix = len * i; ix < len * (i + 1); ++ix)
                 rv.fetch_add(v[ix]);  // <-- sum
        });

    for (auto& t: threads)
        t.join();

    return rv;
}
```

**Вопрос:** как, зная про memory ordering, сделать более быструю реализацию?

<details>
<summary>Ответ</summary>
<p>

```c++
// заменить это:
rv.fetch_add(v[ix]);

// на это:
rv.fetch_add(v[ix], std::memory_order_relaxed);  // <---- no barriers, just atomic update
```

</p>
</details>

<br />

##### lock free stack

На лекции мы реализуем lock free стэк с ошибками на базе списка и разберём ошибки.

Как делать правильный - начните разбирать [с этой статьи](https://habr.com/ru/post/216013/) и далее по ссыкам.

<br />

**Шаг 1:** нарисовать стэк на доске и обсудить как сделать lock-free операции `push`/`pop`

<br />

**Шаг 2:** накидаем примерную реализацию (объяснить каждую строчку)

```c++
template<typename T>
class Stack {
public:
    struct Element
    {
        T data;
        std::atomic<Element*> next{nullptr};
    };
    
private:
    std::atomic<Element*> top;

public:
    bool push(T item)
    {
        Element* val = new Element;
        val->data = std::move(item);
        
        while (true)
        {
            auto* t = top.load();
            val.next.store(t);
            if (top.compare_exchange_strong(t, val))       
               return true;
        }
    }
    
    std::optional<T> pop()
    {
       while (true)
       {
          auto* t = top.load();
          if (!t)
             return std::nullopt ;  // stack is empty

          auto* next = t->next.load();
          if (top.compare_exchange_strong(t, next))
          {
              return /* somehow free memory for element t and return value */;
          }
       }
    }
};
```

**Замечание:** если корректно заменить seq_cst операции на более слабые аналоги с relaxed && acquire/release порядком, то можно получить более быстрый код. Сейчас этим заниматься не будем

**Вопрос:** где в коде ошибка?

<details>
<summary>ответ</summary>
<p>

тут:

```c++
auto* next = t->next.load();
```

Пока выполняли `t->next`, этот элемент из списка могли уладить другим потоком. Нарисовать как её поймать.

Чтобы починить, нужно использовать специальные структуры данных, о них - по ссылке на правильную реализацию

</p>
</details>

**Замечание:** ещё в коде есть ABA-ошибка в методе `pop` в `top.compare_exchange_strong(t, next)`. (Объяснить её)

Как чинить ABA-ошибки, [читайте в работе Страуструпа](http://www.stroustrup.com/isorc2010.pdf).

<br />

**Резюме**:
* вы не хотите писать lock-free алгоритмы в production

<br />

**Неохваченные темы по параллельному программированию:**
* нюансы многопоточности и синхронизаций в boost
* процессы
* fibers
* futexes
* semaphore
* livelock
* thread starving && fair algorithms
* openmp введение
* mpi введение